In [67]:
#%pip install fairlearn
#%pip install lime
#%pip install prophet
#%pip install dalex

In [121]:
# Cargo las librerias necesarias
import pandas 
import numpy as np
import matplotlib.pyplot as plt
import sklearn 
import lime
import fairlearn
from prophet import Prophet
import dalex
from sklearn import preprocessing

In [109]:
# Cargo los datos del dataset train
train = pd.read_table("Data/UH_2023_TRAIN.txt", delimiter="|")

# Hago una copia del dataset por si la necesito después
train_1 = train.copy()

# Observo los datos
train


,CAMPAÑA,ID_FINCA,ID_ZONA,ID_ESTACION,ALTITUD,VARIEDAD,MODO,TIPO,COLOR,SUPERFICIE,PRODUCCION
0,14,76953,515,4,660,26,2,0,1,0.0000,22215.0
1,14,84318,515,4,660,26,2,0,1,0.0000,22215.0
2,14,85579,340,4,520,32,2,0,1,0.0000,20978.0
3,14,69671,340,4,520,32,2,0,1,0.0000,40722.0
4,14,14001,852,14,NaN,81,1,0,1,0.0000,14126.0
...,...,...,...,...,...,...,...,...,...,...,...
9596,22,37461,239,6,700,52,2,0,1,3.6800,NaN
9597,22,58769,239,6,700,32,2,0,1,4.2500,NaN
9598,22,58769,239,6,700,59,2,0,1,4.0700,NaN
9599,22,88928,239,6,700,40,2,0,1,4.5727,NaN


In [116]:
# Convierto los datos a numérico
#train['ALTITUD'].astype(int)
a=train['ALTITUD'].dropna()
a.astype(int)

ValueError: invalid literal for int() with base 10: '650-660'

In [119]:
print(train['ALTITUD'][0:9600])

0           660
1           660
2           520
3           520
4           NaN
         ...   
9595    455-465
9596        700
9597        700
9598        700
9599        700
Name: ALTITUD, Length: 9600, dtype: object


In [99]:
# Eliminamos la columna SUPERFICIE porque no hay datos antes de 2020
train = train.drop(['SUPERFICIE'],axis=1)

In [100]:
# Eliminamos las filas del 2022 porque hay muchos NA y son los que vamos a estimar
train = train.loc[train['CAMPAÑA']!=22]
train

,CAMPAÑA,ID_FINCA,ID_ZONA,ID_ESTACION,ALTITUD,VARIEDAD,MODO,TIPO,COLOR,PRODUCCION
0,14,76953,515,4,660,26,2,0,1,22215.0
1,14,84318,515,4,660,26,2,0,1,22215.0
2,14,85579,340,4,520,32,2,0,1,20978.0
3,14,69671,340,4,520,32,2,0,1,40722.0
4,14,14001,852,14,NaN,81,1,0,1,14126.0
...,...,...,...,...,...,...,...,...,...,...
8521,21,37461,239,6,700,52,2,0,1,28160.1
8522,21,58769,239,6,700,32,2,0,1,41310.0
8523,21,58769,239,6,700,59,2,0,1,45420.0
8524,21,88928,239,6,700,40,2,0,1,56140.0


In [101]:
# Dimensión del dataset
train.shape

(8526, 10)

In [102]:
#train['ALTITUD'] = np.mean[train[ALTITUD]]
mean_altitud = train['ALTITUD'].mean()
train['ALTITUD'].fillna(int(mean_altitud))

TypeError: can only concatenate str (not "int") to str

In [88]:
#comprobamos que sí existen NA
train.isna().any() 

CAMPAÑA        False
ID_FINCA       False
ID_ZONA        False
ID_ESTACION    False
ALTITUD         True
VARIEDAD       False
MODO           False
TIPO           False
COLOR          False
PRODUCCION     False
dtype: bool

In [122]:
train.filter
train_normalized = preprocessing.normalize([train])

ValueError: could not convert string to float: '650-660'

In [7]:
print("lista de variedad:",list(set(list(train['VARIEDAD'])))) #tipo de variedad
total_variedad = train['VARIEDAD'].value_counts().count() #total tipo de variedad
print("total variedad:", total_variedad)

lista de variedad: [4, 8, 9, 10, 15, 17, 23, 26, 32, 38, 40, 43, 48, 52, 55, 59, 62, 65, 68, 71, 81, 84, 87, 92, 94]
total variedad: 25


Tenemos 25 variedades diferentes de vino

In [8]:
print("lista de estacion:",list(set(list(train['ID_ESTACION'])))) #tipo de variedad
total_estacion = train['ID_ESTACION'].value_counts().count() #total tipo de variedad
print("total estacion:", total_estacion)

lista de estacion: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
total estacion: 20


Como podemos ver hay 20 estaciones en el dataset.

In [9]:
media_produccion = train['PRODUCCION'].mean()
print("Producción media:", round(media_produccion, 2))

Producción media: 9209.19


En producción media tenemos 9209.19 vinos / finca.

In [120]:
matrix = train.corr()
print("Correlation matrix is : ")
print(matrix)

Correlation matrix is : 
              CAMPAÑA  ID_FINCA   ID_ZONA  ID_ESTACION  VARIEDAD      MODO  \
CAMPAÑA      1.000000  0.013080  0.009484     0.013470  0.006348  0.196872   
ID_FINCA     0.013080  1.000000 -0.038333    -0.011643  0.007897  0.012798   
ID_ZONA      0.009484 -0.038333  1.000000    -0.164290 -0.009432  0.031552   
ID_ESTACION  0.013470 -0.011643 -0.164290     1.000000  0.058809  0.001483   
VARIEDAD     0.006348  0.007897 -0.009432     0.058809  1.000000 -0.153708   
MODO         0.196872  0.012798  0.031552     0.001483 -0.153708  1.000000   
TIPO         0.020856 -0.004910 -0.079555    -0.099047 -0.056035  0.071646   
COLOR       -0.063086 -0.013785 -0.045611     0.002386 -0.345105  0.083218   
SUPERFICIE   0.460726  0.006650  0.007824     0.034412 -0.018024  0.157849   
PRODUCCION   0.026536  0.042703  0.031858     0.078297 -0.035563  0.271032   

                 TIPO     COLOR  SUPERFICIE  PRODUCCION  
CAMPAÑA      0.020856 -0.063086    0.460726    0.026536  


C:\Users\anabo\AppData\Local\Temp\ipykernel_15496\2135053684.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  matrix = train.corr()
